1. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">A</span> **lab2/6. feladatot** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">módosítsuk a következő módon:</span>

\-Hirdessünk **tranzakciót** a tárolt eljárásban: vagy minden beszúrást hajtson végre a rendszer, vagy egyiket sem!

\-Az adatok adatbázisban való meglétének ellenőrzését, az ID-k tárolását stb. oldjuk meg a lehető legkevesebb lekérdezés segítségével (használjunk lokális változókat)!

\-Minden beszúrás esetén **ellenőrizzük le**, hogy megtörtént-e a beszúrás. Ha nem, hiba(!)üzenet kiíratása mellett térjünk vissza -1-el a tárolt eljárásból (visszatérítési(!) érték).

\-**Fejezzük be minden esetben (pl. feltételes utasítások minden ágán) megfelelően a tranzakciót!**

In [2]:
CREATE OR ALTER PROCEDURE UjKocsma (
    @pKocsmaNev VARCHAR(20),
    @pCim VARCHAR(30),
    @pNyitasDatuma DATE,
    @pBaratNev VARCHAR(30),
    @pBaratEmail VARCHAR(20),
    @pItalTipNev VARCHAR(10),
    @pItalNev VARCHAR(20),
    @pAr REAL
)

AS
BEGIN
    SET NOCOUNT ON
    BEGIN TRY
        BEGIN TRANSACTION

        DECLARE @UjKocsmaID INT
        DECLARE @UjItalID INT
        DECLARE @UjItalTipusID INT

        IF EXISTS (SELECT 1 FROM Kocsmak WHERE Nev = @pKocsmaNev)
        BEGIN
            PRINT 'Ez a kocsma már szerepel az adatbázisban.'
            ROLLBACK TRANSACTION
            RETURN -1
        END
        ELSE
        BEGIN
            
            IF NOT EXISTS (SELECT Nev FROM Baratok WHERE Nev = @pBaratNev)
            BEGIN
                INSERT INTO Baratok(Nev, Email) VALUES (@pBaratNev, @pBaratEmail)
            END

            IF NOT EXISTS (SELECT TipusNev FROM ItalTipusok WHERE TipusNev = @pItalTipNev)
            BEGIN
                INSERT INTO ItalTipusok(TipusNev) VALUES (@pItalTipNev)
            END

            IF NOT EXISTS (SELECT Nev FROM Italok WHERE Nev = @pItalNev)
            BEGIN
                SET @UjItalTipusID = (SELECT TipusID FROM ItalTipusok WHERE TipusNev = @pItalTipNev)
                INSERT INTO Italok(Nev, TipusID) VALUES (@pItalNev, @UjItalTipusID)
            END

            INSERT INTO Kocsmak(Nev, Cim, NyitasDatuma) VALUES (@pKocsmaNev, @pCim, @pNyitasDatuma)
            SET @UjKocsmaID = SCOPE_IDENTITY()

            SET @UjItalID = (SELECT ItalID FROM Italok WHERE Nev = @pItalNev)

            INSERT INTO Arak(Ar, KocsmaID, ItalID) VALUES (@pAr, @UjKocsmaID, @UjItalID)

            COMMIT TRANSACTION
        END
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        PRINT 'Hiba történt a feldolgozás során.';
        RETURN -1;
    END CATCH
END


Commands completed successfully.

Total execution time: 00:00:00.021

In [3]:
EXEC UjKocsma
    @pKocsmanev = 'Planetarium',
    @pCim = 'Cim Plani',
    @pNyitasDatuma = '2018-10-10',
    @pBaratNev = 'Jani',
    @pBaratEmail = 'b.jani005@gmail.com ',
    @pItalTipNev = 'bor',
    @pItalNev = 'rose',
    @pAr = 9

Commands completed successfully.

Total execution time: 00:00:00.057

In [6]:
EXEC UjKocsma
    @pKocsmanev = 'Daisy Geeve',
    @pCim = 'Cim Plani',
    @pNyitasDatuma = '2018-10-10',
    @pBaratNev = 'Jani',
    @pBaratEmail = 'b.jani005@gmail.com ',
    @pItalTipNev = 'bor',
    @pItalNev = 'rose',
    @pAr = 9

Ez a kocsma már szerepel az adatbázisban.

Total execution time: 00:00:00.001

In [4]:
SELECT * FROM Kocsmak
SELECT * FROM Baratok

(12 rows affected)

(17 rows affected)

Total execution time: 00:00:00.116

KocsmaID,Nev,Cim,NyitasDatuma,Bevetel
1,Frederico Sawley,9 Logan Way,2014-12-06,0
2,Orren Broadhurst,87124 Basil Avenue,2016-12-20,0
3,Daisy Geeve,5265 Maple Wood Point,2010-08-26,0
4,Hedwiga Ledbetter,6 Packers Parkway,2012-07-07,0
5,Talia Henrionot,20571 Waxwing Junction,2016-12-07,0
6,Nilson Allflatt,954 Roth Pass,2017-09-16,0
7,Lewes Ashbrook,10 Clyde Gallagher Crossing,2012-10-08,0
8,Sandra Tweed,8 Superior Alley,2011-08-28,0
9,Perrine Chuney,26 Almo Road,2015-10-12,0
10,Sosanna Segeswoeth,4 Cody Park,2013-08-30,0


BaratID,Nev,Tel,Email,Cim
1,Fern Dunphy,6483578626,fdunphy0@princeton.edu,8261 Waxwing Place
2,Dinnie Hebble,1821798103,dhebble1@angelfire.com,76066 Tennessee Road
3,Prisca Dronsfield,8331573181,pdronsfield2@guardian.co.uk,5824 Brown Center
4,Dennie Billison,2911989379,dbillison3@twitter.com,9 Bay Lane
5,Elle Shortell,8449569855,eshortell4@topsy.com,291 Holy Cross Parkway
6,Gipsy Yashunin,6315387706,gyashunin5@csmonitor.com,9 Meadow Ridge Hill
7,Aarika Mashal,5523618089,amashal6@wired.com,288 Lotheville Avenue
8,Celestina Woolaston,9998149794,cwoolaston7@alibaba.gov.cn,934 Kings Center
9,Garvey Chartre,4934922638,gchartre8@alibaba.com,218 Spenser Terrace
10,Rik Turbard,8485977307,rturbard9@vistaprint.com,737 Ramsey Plaza


2.  **a.** **Szúrjuk be** <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">a Kedveltseg mezőt a Kocsmak táblába, ahol Kedveltseg-INT típusú mező, 0-5 közötti értékeket vehet fel, alapértelmezés szerinti értéke 0.</span>

In [7]:
ALTER TABLE Kocsmak
ADD Kedveltseg INT DEFAULT 0 CHECK (Kedveltseg >= 0 AND Kedveltseg <= 5);

Commands completed successfully.

Total execution time: 00:00:00.028

2. **b.** Írjunk **tárolt eljárást**, mely minden kocsma esetén beállítja a Kedveltseg mező értékét annak megfelelően, hogy hány barátunk adott neki 2-nél nagyobb értékelést:

- Barátok száma \< = 4  **(0 is lehet!)** –\> Kedveltseg = 1;
- Barátok száma \> 4 és \<= 10  –\> Kedveltseg = 2;
- Barátok száma \> 10 és \<= 20  –\> Kedveltseg = 3;
- Barátok száma \> 20 és \<= 30  –\> Kedveltseg = 4;
- Barátok száma \> 30  –\> Kedveltseg = 5;

In [12]:
CREATE OR ALTER PROCEDURE KedveltsegBeallitas
AS
BEGIN
    SET NOCOUNT ON
    DECLARE 
    @KocsmaID INT,
    @BaratokSzama INT

    DECLARE kocsmak_cursor CURSOR FOR
    SELECT KocsmaID FROM Kocsmak

    OPEN kocsmak_cursor
    FETCH NEXT FROM kocsmak_cursor INTO @KocsmaID

    WHILE @@FETCH_STATUS = 0
    BEGIN
        SELECT @BaratokSzama = COUNT(*) FROM Kedvencek
        WHERE KocsmaID = @KocsmaID AND Ertekeles > 2

        BEGIN TRANSACTION
        UPDATE Kocsmak
        SET Kedveltseg = 
            CASE
                WHEN @BaratokSzama<=4 THEN 1
                WHEN @BaratokSzama>4 AND @BaratokSzama<=10 THEN 2
                WHEN @BaratokSzama>10 AND @BaratokSzama<=20 THEN 3
                WHEN @BaratokSzama>20 AND @BaratokSzama<=30 THEN 4
                ELSE 5
            END
        WHERE KocsmaID = @KocsmaID

        COMMIT TRANSACTION

        FETCH NEXT FROM kocsmak_cursor INTO @KocsmaID
    END

    CLOSE kocsmak_cursor
    DEALLOCATE kocsmak_cursor
END

Commands completed successfully.

Total execution time: 00:00:00.005

In [10]:
EXEC KedveltsegBeallitas

Commands completed successfully.

Total execution time: 00:00:00.018

In [11]:
SELECT * from Kocsmak

(12 rows affected)

Total execution time: 00:00:00.011

KocsmaID,Nev,Cim,NyitasDatuma,Bevetel,Kedveltseg
1,Frederico Sawley,9 Logan Way,2014-12-06,0,1
2,Orren Broadhurst,87124 Basil Avenue,2016-12-20,0,1
3,Daisy Geeve,5265 Maple Wood Point,2010-08-26,0,1
4,Hedwiga Ledbetter,6 Packers Parkway,2012-07-07,0,1
5,Talia Henrionot,20571 Waxwing Junction,2016-12-07,0,1
6,Nilson Allflatt,954 Roth Pass,2017-09-16,0,1
7,Lewes Ashbrook,10 Clyde Gallagher Crossing,2012-10-08,0,1
8,Sandra Tweed,8 Superior Alley,2011-08-28,0,1
9,Perrine Chuney,26 Almo Road,2015-10-12,0,1
10,Sosanna Segeswoeth,4 Cody Park,2013-08-30,0,1


3\. Írjunk INSERT és DELETE triggert, mely a Kedvencek táblába való beszúrás illetve törlés esetén aktualizálja a Kocsmak tábla Kedveltseg mezőjének értékét! Fontos! Nem elfogadott megoldás: ha a triggeren belül egyszerűen csak meghívjuk a 2. feladatban megírt tárolt eljárást!

**Megj.** Két megoldás is elfogadható: feltételezve, hogy egyszerre (1) csak egy sort, (2) több sort szúrhatunk be/törölhetünk a táblába/táblából! A (2)-es megoldásában bennfoglaltatik az (1)-es megoldása is. **+0.5p:** (2)-es megoldás választása esetén. 

**Megj.** A 3. feladatnak NEM kizáró feltétele a 2. b. alpontban szereplő tárolt eljárás megléte. Ez utóbbi esetben manuálisan adjunk értéket a Kedveltseg mezőnek!

In [13]:
CREATE TRIGGER InsertKedvencek
ON Kedvencek
AFTER INSERT
AS
BEGIN
    SET NOCOUNT ON
    DECLARE 
    @KocsmaID INT, 
    @BaratokSzama INT

    SELECT @KocsmaID=KocsmaID FROM inserted

    SELECT @BaratokSzama=COUNT(*) FROM Kedvencek
    WHERE KocsmaID=@KocsmaID AND Ertekeles>2

    UPDATE Kocsmak
    SET Kedveltseg = 
        CASE
            WHEN @BaratokSzama<=4 THEN 1
            WHEN @BaratokSzama>4 AND @BaratokSzama<=10 THEN 2
            WHEN @BaratokSzama>10 AND @BaratokSzama<=20 THEN 3
            WHEN @BaratokSzama>20 AND @BaratokSzama<=30 THEN 4
            ELSE 5
        END
    WHERE KocsmaID=@KocsmaID
END

Commands completed successfully.

Total execution time: 00:00:00.033

In [14]:
CREATE TRIGGER DeleteKedvencek
ON Kedvencek
AFTER DELETE
AS
BEGIN
    SET NOCOUNT ON
    DECLARE 
    @KocsmaID INT, 
    @BaratokSzama INT;

    SELECT @KocsmaID=KocsmaID FROM deleted

    SELECT @BaratokSzama=COUNT(*) FROM Kedvencek
    WHERE KocsmaID=@KocsmaID AND Ertekeles>2

    UPDATE Kocsmak
    SET Kedveltseg = 
        CASE
            WHEN @BaratokSzama<=4 THEN 1
            WHEN @BaratokSzama>4 AND @BaratokSzama<=10 THEN 2
            WHEN @BaratokSzama>10 AND @BaratokSzama<=20 THEN 3
            WHEN @BaratokSzama>20 AND @BaratokSzama<=30 THEN 4
            ELSE 5
        END
    WHERE KocsmaID=@KocsmaID
END

Commands completed successfully.

Total execution time: 00:00:00.009

In [19]:
INSERT INTO Kedvencek (BaratID,KocsmaID,Ertekeles) VALUES (8,1,5)
INSERT INTO Kedvencek (BaratID,KocsmaID,Ertekeles) VALUES (3,10,5)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.020

In [20]:
SELECT * FROM Kedvencek

(21 rows affected)

Total execution time: 00:00:00.023

BaratID,KocsmaID,Ertekeles
1,1,5
2,2,4
3,1,5
3,4,4
3,7,5
3,10,5
4,1,4
4,9,3
5,1,1
6,9,4


In [21]:
SELECT * FROM Kocsmak

(12 rows affected)

Total execution time: 00:00:00.025

KocsmaID,Nev,Cim,NyitasDatuma,Bevetel,Kedveltseg
1,Frederico Sawley,9 Logan Way,2014-12-06,0,2
2,Orren Broadhurst,87124 Basil Avenue,2016-12-20,0,1
3,Daisy Geeve,5265 Maple Wood Point,2010-08-26,0,1
4,Hedwiga Ledbetter,6 Packers Parkway,2012-07-07,0,1
5,Talia Henrionot,20571 Waxwing Junction,2016-12-07,0,1
6,Nilson Allflatt,954 Roth Pass,2017-09-16,0,1
7,Lewes Ashbrook,10 Clyde Gallagher Crossing,2012-10-08,0,1
8,Sandra Tweed,8 Superior Alley,2011-08-28,0,1
9,Perrine Chuney,26 Almo Road,2015-10-12,0,1
10,Sosanna Segeswoeth,4 Cody Park,2013-08-30,0,1


In [24]:
DELETE FROM Kedvencek WHERE BaratID=3 AND KocsmaID=7
DELETE FROM Kedvencek WHERE BaratID=8 AND KocsmaID=1

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.011

In [25]:
SELECT * FROM Kedvencek
SELECT * FROM Kocsmak

(18 rows affected)

(12 rows affected)

Total execution time: 00:00:00.028

BaratID,KocsmaID,Ertekeles
1,1,5
2,2,4
3,4,4
3,10,5
4,1,4
4,9,3
5,1,1
6,9,4
7,2,5
7,4,2


KocsmaID,Nev,Cim,NyitasDatuma,Bevetel,Kedveltseg
1,Frederico Sawley,9 Logan Way,2014-12-06,0,1
2,Orren Broadhurst,87124 Basil Avenue,2016-12-20,0,1
3,Daisy Geeve,5265 Maple Wood Point,2010-08-26,0,1
4,Hedwiga Ledbetter,6 Packers Parkway,2012-07-07,0,1
5,Talia Henrionot,20571 Waxwing Junction,2016-12-07,0,1
6,Nilson Allflatt,954 Roth Pass,2017-09-16,0,1
7,Lewes Ashbrook,10 Clyde Gallagher Crossing,2012-10-08,0,1
8,Sandra Tweed,8 Superior Alley,2011-08-28,0,1
9,Perrine Chuney,26 Almo Road,2015-10-12,0,1
10,Sosanna Segeswoeth,4 Cody Park,2013-08-30,0,1


4. **a.** Módositsuk az adatbázisunkat a következő módon:

- Keszitsünk egy Eladasok táblát mely a naponta eladott termékeket tartalmazza egy illető Kocsmában: ItalID:Int, KocsmaID:Int, Mennyiseg:Int , Egysegar: float, Datum:Date(aktuális dátum alapértelmezett értékkel) Az elsődleges kulcs legyen a KocsmaID, ItalID, Datum összetett kulcs valamint a KocsmaID és ItalID külső kulcsok.

In [26]:
CREATE TABLE Eladasok (
    ItalID INT,
    KocsmaID INT,
    Mennyiseg INT,
    Egysegar FLOAT,
    Datum DATE DEFAULT GETDATE(),
    PRIMARY KEY (KocsmaID, ItalID, Datum),
    FOREIGN KEY (ItalID) REFERENCES Italok(ItalID),
    FOREIGN KEY (KocsmaID) REFERENCES Kocsmak(KocsmaID)
)

Commands completed successfully.

Total execution time: 00:00:00.012

**b.** <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">&nbsp;Írj tárolt eljárást mely felvezet az Eladások táblába. A tárolt eljárás paraméterei: @KocsmaNev, @ItalNev,&nbsp; @ItalTipusNev, @Mennyiseg, @EgysegAr. A tárolt eljárás ellenőrizze, a kocsma létezését,&nbsp; ha nem létezik hibát jelezve lépjen ki, valamint, hogy az eladott mennyiség pozitív. Ha az&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">@ItalNev</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">&nbsp;vagy a hozzá tartozó</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">@ItalTipusNev típusú ital&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">nem létezik kerüljon felvezetésre az Italok és ItalTipusok táblába valamint a megadott @EgysegAr-al az Arak táblába is. Abban az esetben ha létezik, frissítse az Arak.Ar mezőjét az uj értékkel ha az változott. Az eladás felvezetésekor az eladás Dátuma legyen az aktuális dátum. Végül frissítse a kocsma bevételét hozzáadva a @Mennyiseg * @EgysegAr erteket.</span>

In [31]:
CREATE OR ALTER PROCEDURE EladasokBeirasa(
    @KocsmaNev VARCHAR(30),
    @ItalNev VARCHAR(30),
    @ItalTipusNev VARCHAR(30),
    @Mennyiseg INT,
    @EgysegAr FLOAT
)
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @KocsmaID INT, @ItalID INT, @ItalTipusID INT, @Ar FLOAT
    
    SELECT @KocsmaID = KocsmaID FROM Kocsmak
    WHERE Nev = @KocsmaNev

    IF @KocsmaID IS NULL
    BEGIN
        RAISERROR('Hiba: A kocsma nincs benne az adatbazisban.', 16, 1);
        RETURN
    END
    IF @Mennyiseg<=0
    BEGIN
        RAISERROR ('Hiba: Az eladott mennyiseg nem pozitiv', 16, 2)
        RETURN
    END


    SELECT @ItalTipusID = TipusID FROM ItalTipusok
    WHERE TipusNev = @ItalTipusNev;

    IF @ItalTipusID IS NULL
    BEGIN
        INSERT INTO ItalTipusok (TipusNev) VALUES (@ItalTipusNev)
        SELECT @ItalTipusID=SCOPE_IDENTITY()
    END

    SELECT @ItalID = ItalID FROM Italok
    WHERE Nev =@ItalNev AND TipusID= @ItalTipusID;

    IF @ItalID IS NULL
    BEGIN
        INSERT INTO Italok (Nev, TipusID) VALUES (@ItalNev, @ItalTipusID)
        SELECT @ItalID=SCOPE_IDENTITY()
    END

    SELECT @Ar =Ar FROM Arak
    WHERE KocsmaID= @KocsmaID AND ItalID = @ItalID

    IF @Ar IS NULL OR @Ar <> @EgysegAr
    BEGIN
        IF @Ar IS NULL
        BEGIN
            INSERT INTO Arak (KocsmaID, ItalID, Ar) VALUES (@KocsmaID, @ItalID, @EgysegAr)
        END
        ELSE
        BEGIN
            UPDATE Arak
            SET Ar = @EgysegAr WHERE KocsmaID =@KocsmaID AND ItalID = @ItalID
        END
    END

    INSERT INTO Eladasok (ItalID, KocsmaID, Mennyiseg, Egysegar) VALUES (@ItalID, @KocsmaID, @Mennyiseg, @EgysegAr)


    UPDATE Kocsmak
    SET Bevetel = Bevetel+(@Mennyiseg*@EgysegAr) WHERE KocsmaID = @KocsmaID
END

   


Commands completed successfully.

Total execution time: 00:00:00.016

In [30]:
SELECT Nev, KocsmaID FROM Kocsmak
SELECT Nev, ItalID FROM Italok
SELECT * FROM ItalTipusok
SELECT * FROM Arak

(12 rows affected)

(11 rows affected)

(6 rows affected)

(30 rows affected)

Total execution time: 00:00:00.043

Nev,KocsmaID
Frederico Sawley,1
Orren Broadhurst,2
Daisy Geeve,3
Hedwiga Ledbetter,4
Talia Henrionot,5
Nilson Allflatt,6
Lewes Ashbrook,7
Sandra Tweed,8
Perrine Chuney,9
Sosanna Segeswoeth,10


Nev,ItalID
Valenties,1
Alexandrion,2
Absolute,3
Alexander,4
V33,5
Tatra tea,6
Coca Cola,7
Banderas,8
Fekete tea,9
Kinley,10


TipusID,TipusNev
6,bor
3,konyak
4,tea
5,tonic
2,vodka
1,whiskey


KocsmaID,ItalID,Ar
1,1,8
1,2,16
1,4,9
1,5,6
1,8,45
1,9,25
1,10,29
2,2,38
2,7,39
2,10,32


In [34]:
DECLARE 
    @KocsmaNev VARCHAR(30) = (SELECT Nev FROM Kocsmak WHERE KocsmaID=4),
    @ItalNev VARCHAR(30) = (SELECT Nev FROM Italok WHERE ItalID=11),
    @ItalTipusNev VARCHAR(30) = (SELECT TipusNev FROM ItalTipusok WHERE TipusID=6),
    @Mennyiseg INT = 3,
    @EgysegAr FLOAT = 10.5
EXEC EladasokBeirasa @KocsmaNev, @ItalNev, @ItalTipusNev, @Mennyiseg, @EgysegAr 

Commands completed successfully.

Total execution time: 00:00:00.031

In [36]:
SELECT * From Eladasok
SELECT Nev, KocsmaID FROM Kocsmak
SELECT Nev, ItalID FROM Italok
SELECT * FROM ItalTipusok
SELECT * FROM Arak

(1 row affected)

(12 rows affected)

(11 rows affected)

(6 rows affected)

(31 rows affected)

...

ItalID,KocsmaID,Mennyiseg,Egysegar,Datum
11,4,3,10.5,2024-01-06


Nev,KocsmaID
Frederico Sawley,1
Orren Broadhurst,2
Daisy Geeve,3
Hedwiga Ledbetter,4
Talia Henrionot,5
Nilson Allflatt,6
Lewes Ashbrook,7
Sandra Tweed,8
Perrine Chuney,9
Sosanna Segeswoeth,10


Nev,ItalID
Valenties,1
Alexandrion,2
Absolute,3
Alexander,4
V33,5
Tatra tea,6
Coca Cola,7
Banderas,8
Fekete tea,9
Kinley,10


TipusID,TipusNev
6,bor
3,konyak
4,tea
5,tonic
2,vodka
1,whiskey


KocsmaID,ItalID,Ar
1,1,8
1,2,16
1,4,9
1,5,6
1,8,45
1,9,25
1,10,29
2,2,38
2,7,39
2,10,32


In [37]:
EXEC EladasokBeirasa 'Bleriot', 'Hugo', 'pezsgo', 2, 40 

: Msg 50000, Level 16, State 1, Procedure EladasokBeirasa, Line 19
Hiba: A kocsma nincs benne az adatbazisban.

Total execution time: 00:00:00.020

In [39]:
EXEC EladasokBeirasa 'Sosanna Segeswoeth', 'Hugo', 'pezsgo', 2, 40 

Commands completed successfully.

Total execution time: 00:00:00.026

In [40]:
SELECT * From Eladasok
SELECT Nev, KocsmaID FROM Kocsmak
SELECT Nev, ItalID FROM Italok
SELECT * FROM ItalTipusok
SELECT * FROM Arak

(2 rows affected)

(12 rows affected)

(12 rows affected)

(7 rows affected)

(32 rows affected)

...

ItalID,KocsmaID,Mennyiseg,Egysegar,Datum
11,4,3,10.5,2024-01-06
12,10,2,40,2024-01-06


Nev,KocsmaID
Frederico Sawley,1
Orren Broadhurst,2
Daisy Geeve,3
Hedwiga Ledbetter,4
Talia Henrionot,5
Nilson Allflatt,6
Lewes Ashbrook,7
Sandra Tweed,8
Perrine Chuney,9
Sosanna Segeswoeth,10


Nev,ItalID
Valenties,1
Alexandrion,2
Absolute,3
Alexander,4
V33,5
Tatra tea,6
Coca Cola,7
Banderas,8
Fekete tea,9
Kinley,10


TipusID,TipusNev
6,bor
3,konyak
7,pezsgo
4,tea
5,tonic
2,vodka
1,whiskey


KocsmaID,ItalID,Ar
1,1,8
1,2,16
1,4,9
1,5,6
1,8,45
1,9,25
1,10,29
2,2,38
2,7,39
2,10,32


In [41]:
EXEC EladasokBeirasa 'Sosanna Segeswoeth', 'Hugo', 'pezsgo', -4, 40 

: Msg 50000, Level 16, State 2, Procedure EladasokBeirasa, Line 24
Hiba: Az eladott mennyiseg nem pozitiv

Total execution time: 00:00:00.018